In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import subprocess
import numpy as np
import glob

In [ ]:
def savefig(name, **kwargs):
    fn = "{}.pdf".format(name)
    pp = PdfPages(fn)
    pp.savefig(dpi=1200, bbox_inches="tight", **kwargs)
    pp.close()
    print("saving and cropping ", fn)
    subprocess.run(["pdfcrop", fn, fn])

In [ ]:
# us -> MHz
reference_frequency_per_time_unit = 2.0E3
TRANSITION_LATENCY = "Transition latency [µs]"
WAIT_LATENCY = "Time since last frequency switch [µs]"
SOURCE_FREQUENCY = "Source Frequency [GHz]"
TARGET_FREQUENCY = "Target Frequency [GHz]"
FREQ_TRANSITION = "Frequency transition [GHz]"

In [ ]:
folder = "results/hati"
files_matcher = f"./{folder}/*.txt"

processed_data = pd.DataFrame()
all_data = pd.DataFrame()

for file in glob.glob(files_matcher):
    frequencies, slug = os.path.basename(file).split('-')
    start, target = frequencies.split('_')
    start_ghz = int(start) / 1e6
    target_ghz = int(target) / 1e6
    try:
        data = pd.read_csv(file, sep='\t',comment='#')
        print(data)
    except:
        continue
    data=data.rename(columns={"Change time (with write) [cycles]": TRANSITION_LATENCY, "Time since last frequency change request [cycles]": WAIT_LATENCY})
    data[TRANSITION_LATENCY] = data[TRANSITION_LATENCY] / reference_frequency_per_time_unit
    data[WAIT_LATENCY] = data[WAIT_LATENCY] / reference_frequency_per_time_unit

    # filter invalid measurement values
    data=data[data[TRANSITION_LATENCY]>0]

    processed_data = pd.concat([processed_data, pd.DataFrame([
        {'valids': len(data),
         'median': np.median(data[TRANSITION_LATENCY].quantile(0.5)),
         '1percentQuantile': data[TRANSITION_LATENCY].quantile(0.01),
         '99percentQuantile': data[TRANSITION_LATENCY].quantile(0.99),
         SOURCE_FREQUENCY: start_ghz,
         TARGET_FREQUENCY: target_ghz}
    ])])

    data[SOURCE_FREQUENCY] = start_ghz
    data[TARGET_FREQUENCY] = target_ghz
    data[FREQ_TRANSITION] = f"{start_ghz}GHz -> {target_ghz}GHz"
    all_data = pd.concat([all_data, data])

processed_data

In [ ]:
all_data.reset_index()
all_data

In [ ]:
all_data_random_sample = all_data.sample(frac=.10)
all_data_random_sample

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="valids"), vmin=0, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()
savefig(f"{folder}/ftalat_valids")

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="median"), vmin=0, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()
savefig(f"{folder}/ftalat_median")

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="1percentQuantile"), vmin=0, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()
savefig(f"{folder}/ftalat_1percentQuantile")

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="99percentQuantile"), vmin=0, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()
savefig(f"{folder}/ftalat_99percentQuantile")

In [ ]:
row_order = list(reversed(sorted(all_data[SOURCE_FREQUENCY].unique())))
col_order = sorted(list(all_data[TARGET_FREQUENCY].unique()))
print(row_order, col_order)

In [ ]:
g = sns.FacetGrid(all_data_random_sample, col=TARGET_FREQUENCY,  row=SOURCE_FREQUENCY, height=5, row_order=row_order, col_order=col_order)
g.map(sns.scatterplot, WAIT_LATENCY, TRANSITION_LATENCY)
g.add_legend()
savefig(f"{folder}/ftalat_scatter_wait_transition_latency")

In [ ]:
g = sns.FacetGrid(all_data_random_sample, col=TARGET_FREQUENCY, row=SOURCE_FREQUENCY, height=5, row_order=row_order, col_order=col_order)
g.map(sns.histplot, TRANSITION_LATENCY, binwidth=10)
g.add_legend()
savefig(f"{folder}/ftalat_distribution_transition_latency")